In [1]:
"""detect lip motion"""
import cv2
import matplotlib.pyplot as plt
import numpy as np

import os
import glob
import tqdm as tqdm
import json
import zipfile


from sklearn import svm
from sklearn.metrics import pairwise_distances

In [ ]:
floder = "../data"
output_path = "full_data-17-11.zip"

In [2]:
data = []
label = []
for path in glob.glob(f"motion/*"):
    with open(path, "r", encoding='utf-8') as f:
        lips = json.load(f)
    diffs = np.diff(lips, axis=0)

    for diff in diffs:
        data.append(diff)
        label.append(1)
        

In [3]:
for path in glob.glob(f"unmotion/*"):
    diffs = []
    with open(path, "r", encoding='utf-8') as f:
        lips = json.load(f)
    diffs = np.diff(lips, axis=0)
    for diff in diffs:
        data.append(diff)
        label.append(0)

In [4]:
clf = svm.SVC()
clf.fit(data, label)

SVC()

In [16]:
motion_datas = []
for num in tqdm.tqdm(range(1, 11)):
    for videos in glob.glob(f"{floder}/Collect_{num}/lip/lipali-new/*"):
        for path in glob.glob(f"{videos}/*"):
            predicts = []
            with open(path, "r", encoding='utf-8') as f:
                lips = json.load(f)
            diffs = np.diff(lips, axis=0)
            
            for diff in diffs:
                
                predict = clf.predict(diff.reshape(-1, 20))
                predicts.extend(predict)
            if set(predicts) == {1}:
                motion_datas.append(path)

100%|██████████| 10/10 [1:04:34<00:00, 387.40s/it]


In [102]:
len(motion_datas)

58665

In [53]:
count = 0
with zipfile.ZipFile(output_path, 'a', zipfile.ZIP_DEFLATED) as zipf:
    for motion_data in motion_datas:
        Collect = motion_data.split("/")[0]
        floder = motion_data.split("\\")[1]
        filename = f"{os.path.basename(motion_data)[:-5]}.mp4"

        path_1 = f"{Collect}/lip/video-fpsfree-new/{floder}/{filename}"
        path_2 = f"{Collect}/script/{floder}/video/{filename}"

        try:
            # arcname_1 = os.path.relpath(path_1)
            zipf.write(path_1)
            # arcname_2 = os.path.relpath(path_2)
            zipf.write(path_2)
        except:
            count+=1
            print(path_1)
            if count == 10:
                break

Collect_1/lip/video-fpsfree-new/video-10016/190-ตอนนี้เราจับที่สี.mp4
